# Simple usage of Dask vectors and Dask operators

@Author: Ettore Biondi - ettore88@stanford.edu

In this notebook, we describe the usage of the Dask-based classes. These objects are designed to take advantage of computational power of computer clusters composed of multiple nodes. To this end, we employ the existing classes in combination of Dask (https://dask.org/). We show the syntax with which a user can instantiate Dask-based objects from existing constructors using a local Dask cluster. The same syntax applies to the other supported Dask clusters.

### Importing necessary libraries

In [1]:
import numpy as np
import occamypy
#Plotting library
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
# %matplotlib inline
params = {
    'image.interpolation': 'nearest',
    'image.cmap': 'gray',
    'savefig.dpi': 300,  # to adjust notebook inline plot size
    'axes.labelsize': 14, # fontsize for x and y labels (was 10)
    'axes.titlesize': 14,
    'font.size': 14,
    'legend.fontsize': 14,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'text.usetex':False
}
matplotlib.rcParams.update(params)

WARNING! DATAPATH not found. The folder /tmp will be used to write binary files


### Starting a Dask cluster and client
Let's start by starting a local Dask client and show how to get some information from such object. We are going to start 4 workers.

In [2]:
help(occamypy.DaskClient)

Help on class DaskClient in module occamypy.dask.utils:

class DaskClient(builtins.object)
 |  DaskClient(**kwargs)
 |  
 |  Class useful to construct a Dask Client to be used with Dask vectors and operators
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      Constructor for obtaining a client to be used when Dask is necessary
 |      1) Cluster with shared file system and ssh capability:
 |      :param hostnames : - list; list of strings containing the host names or IP addresses of the machines that
 |      the user wants to use in their cluster/client (First hostname will be running the scheduler!) [None]
 |      :param scheduler_file_prefix : string; prefix to used to create dask scheduler-file.
 |      :param logging : - boolean; Logging scheduler and worker stdout to files within dask_logs folder [True]
 |      Must be a mounted path on all the machines. Necessary if hostnames are provided [$HOME/scheduler-]
 |      2) Local cluster:
 |      :param local_param

In [3]:
client_params = {"processes":True}
client = occamypy.DaskClient(local_params=client_params, n_wrks=4)

In [4]:
print("Number of workers = %d" % client.getNworkers())
print("Workers Ids = %s" % client.getWorkerIds())

Number of workers = 4
Workers Ids = ['tcp://127.0.0.1:34099', 'tcp://127.0.0.1:35621', 'tcp://127.0.0.1:44235', 'tcp://127.0.0.1:44345']


### Dask vectors
Now that we have a Dask client, we can instantiate vectors using the Dask interface. The currently supported methods to create such objects are the following:
1. Instantiate a vector template and spread it using the chunk parameter
2. Instantiate multiple vectors and spreading them to the given workers

In [6]:
# Method 1
vec_temp = occamypy.VectorNumpy((200, 300))
chunks = (3, 4, 6, 2) # 3 vectors to worker 1; 4 vectors to worker 2; ...
vecD = occamypy.DaskVector(client, vector_template=vec_temp, chunks=chunks)

vecD inherits all the methods from the abstract vector class. Let's try some of them.

In [7]:
# shape
print("List of shapes: %s" % vecD.shape)
# Randomize
vecD.rand()
# Norm
print("Dask vector norm = %s" % vecD.norm())
# Scaling
vecD.scale(10)
print("Scaled Dask vector norm = %s" % vecD.norm())
# Cloning
vecD1 = vecD.clone()
# Summing two vectors
vecD1 + vecD
# Check norm
print("Sum Dask vector norm = %s" % vecD1.norm())

List of shapes: [(200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300), (200, 300)]
Dask vector norm = 547.4540265821655
Scaled Dask vector norm = 5474.540265821655
Sum Dask vector norm = 10949.08053164331


The Dask vector contains a list of the future objects pointing to the vector chunks. Let's see how to see which worker has a given chunk.

In [8]:
print("Future object to first chunk: %s" % vecD.vecDask[0])
print("Worker having given chunk: %s" % client.getClient().who_has(vecD.vecDask[0]))

Future object to first chunk: <Future: finished, type: occamypy.VectorNumpy, key: _call_clone-73f32d2b-17a7-4969-b8e2-5efb1c0f4a45>
Worker having given chunk: {'_call_clone-73f32d2b-17a7-4969-b8e2-5efb1c0f4a45': ('tcp://127.0.0.1:34099',)}


Let's now create a vector using a different Dask-vector constructor. Here, we instantiate all the chunks and then spread them onto the given workers.

In [10]:
vec1 = occamypy.VectorNumpy((200, 300))
vec2 = occamypy.VectorNumpy((10, 30))
vec3 = occamypy.VectorNumpy((250, 1))
# We use the parameter chunks to select which worker will have a given vector instance
vecD = occamypy.DaskVector(client, vectors=[vec1, vec2, vec3], chunks=(1, 1, 0, 1))

Let's try similar tests as before.

In [14]:
# shape
print("List of shapes: %s" % vecD.shape)
# Randomize
vecD.rand()
# Norm
print("Dask vector norm = %s" % vecD.norm())
# Scaling
vecD.scale(10)
print("Scaled Dask vector norm = %s" % vecD.norm())
# Cloning
vecD1 = vecD.clone()
# Summing two vectors
vecD1 + vecD
# Check norm
print("Sum Dask vector norm = %s" % vecD1.norm())
print("Future object to third chunk: %s" % vecD.vecDask[2])
print("Worker having given chunk: %s" % client.getClient().who_has(vecD.vecDask[2]))

List of shapes: [(200, 300), (10, 30), (250, 1)]
Dask vector norm = nan
Scaled Dask vector norm = nan
Sum Dask vector norm = nan
Future object to third chunk: <Future: finished, type: occamypy.VectorNumpy, key: VectorNumpy-1b6bf253d8bf8366a028d18846d3f9a5>
Worker having given chunk: {'VectorNumpy-1b6bf253d8bf8366a028d18846d3f9a5': ('tcp://127.0.0.1:44345',)}


### Dask operators
Now, let's try to instantiate Dask operators. These kind of objects are pretty useful when large-scale problems have to be solved. The main idea behind the interface is to pass a given operator constructor and the necessary parameters so that the object is directly instantiated within the Dask workers of a client.

In [17]:
# Construct a simple scaling operator acting on each chunk of a Dask Vector
vec = occamypy.VectorNumpy((100, 25))
chunks = (2, 3, 5, 10)
sc = 10.0
vecD = occamypy.DaskVector(client, vector_template=vec, chunks=chunks)
# Creating list of lists of the arguments for the operator's constructor
scal_op_args = [(vec_i, sc) for vec_i in vecD.vecDask]

# Instantiating Dask operator
scaleOpD = occamypy.DaskOperator(client, occamypy.Scaling, scal_op_args, chunks)

Similarly to the Dask vector class, a Dask operator object inherits all the methods from the corresponding abstract class. Let's try some of those methods.

In [18]:
# Dot-product test
scaleOpD.dotTest(True)
# Power method
max_eig = scaleOpD.powerMethod()
print("\nMaximum eigenvalue = %s" % max_eig)

Dot-product tests of forward and adjoint operators
--------------------------------------------------
Applying forward operator add=False
 Runs in: 0.09250855445861816 seconds
Applying adjoint operator add=False
 Runs in: 0.07437300682067871 seconds
Dot products add=False: domain=1.254629e+02 range=1.254629e+02 
Absolute error: 1.421085e-13
Relative error: 1.132674e-15 

Applying forward operator add=True
 Runs in: 0.07640290260314941 seconds
Applying adjoint operator add=True
 Runs in: 0.08943676948547363 seconds
Dot products add=True: domain=2.509258e+02 range=2.509258e+02 
Absolute error: 3.410605e-13
Relative error: 1.359209e-15 

-------------------------------------------------

Maximum eigenvalue = 10.000000000000002


Let's now try to apply this Dask operator.

In [19]:
vecD.rand()
vecD1 = scaleOpD.getRange().clone()
scaleOpD.forward(False, vecD, vecD1)
print("Norm of the input = %s" % vecD.norm())
print("Norm of the output = %s" % vecD1.norm())

Norm of the input = 128.97214922487566
Norm of the output = 1289.7214922487565


Finally, let's combine an operator that spreads and collects a local vector onto a Dask-vector chunks. Such operator is useful when the same vector is employed multiple times on different operators embarrassingly-parallelizable.

In [20]:
S = occamypy.DaskSpread(client, vec, chunks)
S.dotTest(True) # checking dot-product

Dot-product tests of forward and adjoint operators
--------------------------------------------------
Applying forward operator add=False
 Runs in: 0.2857086658477783 seconds
Applying adjoint operator add=False
 Runs in: 0.07837414741516113 seconds
Dot products add=False: domain=1.266344e+02 range=1.266344e+02 
Absolute error: 4.263256e-14
Relative error: 3.366587e-16 

Applying forward operator add=True
 Runs in: 0.2619023323059082 seconds
Applying adjoint operator add=True
 Runs in: 0.073211669921875 seconds
Dot products add=True: domain=2.532688e+02 range=2.532688e+02 
Absolute error: 8.526513e-14
Relative error: 3.366587e-16 

-------------------------------------------------


In [21]:
#Chain of scaling and spreading operator
scale_S = scaleOpD * S
scale_S.dotTest(True) # checking dot-product
# Testing product of Dask Operators
x = vec.rand()
y = scale_S.getRange().clone()
scale_S.forward(False, x, y)
print("\nFirst element of x = %s" % x.getNdArray()[0,0])
print("First element of y = %s" % y.getNdArray()[0][0,0])

Dot-product tests of forward and adjoint operators
--------------------------------------------------
Applying forward operator add=False
 Runs in: 0.39472270011901855 seconds
Applying adjoint operator add=False
 Runs in: 0.21671438217163086 seconds
Dot products add=False: domain=5.248635e+02 range=5.248635e+02 
Absolute error: 1.136868e-13
Relative error: 2.166027e-16 

Applying forward operator add=True
 Runs in: 0.3732430934906006 seconds
Applying adjoint operator add=True
 Runs in: 0.17659807205200195 seconds
Dot products add=True: domain=1.049727e+03 range=1.049727e+03 
Absolute error: 2.273737e-13
Relative error: 2.166027e-16 

-------------------------------------------------


/nas/home/fpicetti/occamypy/occamypy/numpy/vector.py:73: RuntimeWarning: overflow encountered in square
  rms = np.sqrt(np.mean(np.square(self.getNdArray())))



First element of x = nan
First element of y = nan
